In [13]:
import pandas as pd
import requests
import bs4 as beautifulsoup
from bs4 import BeautifulSoup
#https://www.nbadraft.net/search-players/ Website we will iterate through

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

base_url = 'http://www.nbadraft.net'


In [45]:
def FetchURL(url):
    #print(url)
    #raise Exception("Stop")
    r = requests.get(url)
    return r.content

In [66]:
url_list = []


#Creating a list of all links to pages where tables of player links are stored

for i in range(10):
    url_link = base_url + "/search-players/"
    if i == 0:
        url_list.append(url_link)
    else: 
        url_link = url_link + "?offset="+str(i)
        url_list.append(url_link)
    
#https://www.nbadraft.net/search-players/ ~ first page
#https://www.nbadraft.net/search-players/?offset=210 ~ all other pages

player_links = []

In [11]:
url_list[:5]

['http://www.nbadraft.net/search-players/',
 'http://www.nbadraft.net/search-players/?offset=1',
 'http://www.nbadraft.net/search-players/?offset=2',
 'http://www.nbadraft.net/search-players/?offset=3',
 'http://www.nbadraft.net/search-players/?offset=4']

In [69]:
def GetPlayerURLs(url):
    #Creates the set of player urls of a single page
    soup = BeautifulSoup(FetchURL(url), "html.parser")

    player_list = []

    for a in soup.find_all(href=True):
        #print("Found the URL:", a["href"], a.text)
        if "https://www.nbadraft.net/players/" in a["href"]:
            player_list.append(a["href"])

    player_set = set(player_list) 
    
    return player_set

def AllPlayerURLs(all_urls=url_list):
    """
    Creates the set of all player urls, from all pages
    """
    complete_set = set()
    
    count_page = 0 
    for url in all_urls:
        print("Scraping page #:", count_page)
        complete_set.update(GetPlayerURLs(url))
        count_page+=1
    
    
    return complete_set
    

player_url_list = list(AllPlayerURLs())

Scraping page #: 0
Scraping page #: 1
Scraping page #: 2
Scraping page #: 3
Scraping page #: 4
Scraping page #: 5
Scraping page #: 6
Scraping page #: 7
Scraping page #: 8
Scraping page #: 9


In [83]:
mydf = pd.DataFrame(columns=["Height", "Weight"])

In [84]:
mydf

,Height,Weight


In [93]:
def ScrapeProfile(url, df=mydf):
    soup = BeautifulSoup(FetchURL(url), "html.parser")
    count=0
    profile_data = []
    for i in soup.find_all('div', {'class': 'wpb_wrapper'}):
        player_soup = list(filter(None, i.text.split("\n")))
        print(player_soup, count)
        #profile_data.append(i.text)
        df = df.append({"Height": player_soup[1], "Weight":player_soup[2]},ignore_index=True)
        count+=1
        return df
    
        raise Exception()
        
    return profile_data
    
ScrapeProfile("https://www.nbadraft.net/players/abdoulaye-loum/")

['9 - Abdoulaye Loum ', '6-10,', '230', 'Power Forward/Center', 'France ', 'International  ', 'Birthday ', '04/03/91 (29.1 yrs)', 'Hometown ', 'Mont-de-Marsan, France ', 'Team Site Profile ', 'Team Site Profile', 'Statistics ', 'Statistics', 'Athleticism ', '8 ', 'Size ', '9 ', 'Defense ', '7 ', 'Strength ', '7 ', 'Quickness ', '8 ', 'Leadership ', '6 ', 'Jump Shot ', '7 ', 'NBA Ready ', '6 ', 'Rebounding ', '7 ', 'Potential ', '7 ', 'Post Skills ', '6 ', 'Intangibles ', '6 ', '84', 'Overall:'] 0


,Height,Weight
0,"6-10,",230


In [ ]:
for pages in url_list:
    for player_link in player_links:
        ScrapeProfile()